# 🚀 02 - Carregando Modelos com Propósito
## ⚖️ Do Gerador de Texto ao Assistente Jurídico

**Objetivo:** Aprender a carregar e usar modelos especializados para aplicações jurídicas reais.

---

## 📋 Lições do Notebook Anterior
1. ✅ **GPT-2 Small Portuguese** funciona na sua GPU (2.1 GB VRAM)
2. ❌ **Phi-2 (2.7B)** é grande demais mesmo com 4-bit
3. ⚠️  Modelos base "alucinam" fatos jurídicos
4. 💡 **Solução:** Usar modelos especializados + técnicas de _prompt_

---

## 🎯 Objetivos Deste Notebook
1. Carregar diferentes tipos de modelos (generativo vs encoder)
2. Usar modelos jurídicos especializados em português
3. Aplicar _device_map="auto"_ para gerenciamento inteligente de memória
4. Criar um pipeline simples de análise contratual

---

## ⚡ Estratégia para GPU 2GB
- **Modelos ≤ 500M parâmetros:** Carregamento direto
- **Modelos ≤ 2B parâmetros:** Necessitam otimização
- **device_map="auto":** Distribui camadas automaticamente entre GPU/CPU
- **Sequências curtas:** ≤ 512 tokens para análise jurídica

In [1]:
# Configuração e Diagnóstico de Memória
import torch
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModelForMaskedLM,
    AutoModelForSequenceClassification
)
import gc
import psutil
import os

print("="*70)
print("🧮 DIAGNÓSTICO INICIAL DO SISTEMA")
print("="*70)

# Informações da GPU
print(f"🔧 PyTorch: {torch.__version__}")
print(f"🎮 GPU disponível: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    vram_total = torch.cuda.get_device_properties(0).total_memory / 1e9
    vram_alocada = torch.cuda.memory_allocated() / 1e9
    vram_livre = vram_total - vram_alocada
    
    print(f"💻 GPU: {gpu_name}")
    print(f"💾 VRAM Total: {vram_total:.1f} GB")
    print(f"💾 VRAM Alocada: {vram_alocada:.2f} GB")
    print(f"💾 VRAM Livre: {vram_livre:.2f} GB")
    
    # Recomendações baseadas na memória livre
    if vram_livre > 1.5:
        print("✅ Ótimo! Pode carregar modelos até ~500M parâmetros")
    elif vram_livre > 0.8:
        print("⚠️  Cuidado! Limite-se a modelos ≤ 300M parâmetros")
    else:
        print("❌ Memória muito baixa! Libere memória antes de continuar")

# Informações da RAM
ram = psutil.virtual_memory()
print(f"\n🧠 RAM Total: {ram.total / 1e9:.1f} GB")
print(f"🧠 RAM Disponível: {ram.available / 1e9:.1f} GB")

print("\n" + "="*70)

🧮 DIAGNÓSTICO INICIAL DO SISTEMA
🔧 PyTorch: 2.3.1+cu118
🎮 GPU disponível: True
💻 GPU: NVIDIA GeForce 930M
💾 VRAM Total: 2.1 GB
💾 VRAM Alocada: 0.00 GB
💾 VRAM Livre: 2.15 GB
✅ Ótimo! Pode carregar modelos até ~500M parâmetros

🧠 RAM Total: 10.6 GB
🧠 RAM Disponível: 4.6 GB



In [2]:
# Tipos Fundamentais de Modelos - Teoria Prática
print("🤔 ENTENDENDO OS TIPOS DE MODELOS")
print("="*70)

tipos_modelos = [
    {
        "tipo": "Modelos Generativos (Decoder-only)",
        "exemplo": "GPT-2, GPT-3, LLaMA, Phi-2",
        "função": "Gerar texto sequencialmente",
        "uso": "Redação, conclusão, criatividade",
        "carregamento": "AutoModelForCausalLM",
        "para_sua_gpu": "✅ GPT-2 Small PT funciona bem"
    },
    {
        "tipo": "Modelos Encoder (BERT-like)",
        "exemplo": "BERT, RoBERTa, Legal-BERT",
        "função": "Compreensão/classificação de texto",
        "uso": "Análise, busca, NER, Q&A",
        "carregamento": "AutoModelForMaskedLM ou AutoModelForSequenceClassification",
        "para_sua_gpu": "✅ BERTimbau e variantes cabem"
    },
    {
        "tipo": "Modelos Encoder-Decoder",
        "exemplo": "T5, BART",
        "função": "Transformação de texto",
        "uso": "Tradução, sumarização, reformulação",
        "carregamento": "AutoModelForSeq2SeqLM",
        "para_sua_gpu": "⚠️  T5-small pode caber, mas testar"
    }
]

print("\n🎯 RESUMO PARA SUA APLICAÇÃO JURÍDICA:")
print("1. Para GERAR texto (petições, cláusulas): Use modelos generativos")
print("2. Para ANALISAR texto (contratos, leis): Use modelos encoder (BERT-like)")
print("3. Para RESUMIR/REFORMULAR: Use encoder-decoder (se couber na memória)")

print("\n📋 COMPARAÇÃO PRÁTICA:")
for i, tipo in enumerate(tipos_modelos, 1):
    print(f"\n{i}. {tipo['tipo']}")
    print(f"   📝 Exemplo: {tipo['exemplo']}")
    print(f"   🎯 Uso principal: {tipo['uso']}")
    print(f"   💻 Para sua GPU: {tipo['para_sua_gpu']}")

🤔 ENTENDENDO OS TIPOS DE MODELOS

🎯 RESUMO PARA SUA APLICAÇÃO JURÍDICA:
1. Para GERAR texto (petições, cláusulas): Use modelos generativos
2. Para ANALISAR texto (contratos, leis): Use modelos encoder (BERT-like)
3. Para RESUMIR/REFORMULAR: Use encoder-decoder (se couber na memória)

📋 COMPARAÇÃO PRÁTICA:

1. Modelos Generativos (Decoder-only)
   📝 Exemplo: GPT-2, GPT-3, LLaMA, Phi-2
   🎯 Uso principal: Redação, conclusão, criatividade
   💻 Para sua GPU: ✅ GPT-2 Small PT funciona bem

2. Modelos Encoder (BERT-like)
   📝 Exemplo: BERT, RoBERTa, Legal-BERT
   🎯 Uso principal: Análise, busca, NER, Q&A
   💻 Para sua GPU: ✅ BERTimbau e variantes cabem

3. Modelos Encoder-Decoder
   📝 Exemplo: T5, BART
   🎯 Uso principal: Tradução, sumarização, reformulação
   💻 Para sua GPU: ⚠️  T5-small pode caber, mas testar


In [3]:
# device_map="auto" - O Superpoder para GPU Limitada
print("\n" + "="*70)
print("⚡ DEVICE_MAP='AUTO': Seu Aliado para 2GB")
print("="*70)

print("""
O que é device_map="auto"?
• Distribui automaticamente as camadas do modelo entre GPU, CPU e até disco
• Otimiza o uso de memória baseado no hardware disponível
• Para sua GPU 2GB: Mantém camadas críticas na GPU, offloada o resto

COMO FUNCIONA NA PRÁTICA:
1. Camadas mais usadas (atenção) → GPU
2. Camadas menos usadas (algumas feed-forward) → CPU  
3. Se necessário, algumas camadas → Disco (mais lento)

VANTAGENS PARA VOCÊ:
• Carrega modelos maiores que não caberiam totalmente na GPU
• Balanceamento automático performance/memória
• Configuração zero - só passar o parâmetro!
""")

# Demonstração visual
print("\n🎯 EXEMPLO DE DISTRIBUIÇÃO PARA UM BERT-BASE (110M):")
distribuicao_exemplo = {
    "GPU (2.1 GB)": "70% das camadas (atenção + algumas FFN)",
    "CPU (RAM)": "25% das camadas (FFN menos críticas)",
    "Disco (opcional)": "5% (se RAM também estiver limitada)"
}

for local, percentual in distribuicao_exemplo.items():
    print(f"   • {local}: {percentual}")


⚡ DEVICE_MAP='AUTO': Seu Aliado para 2GB

O que é device_map="auto"?
• Distribui automaticamente as camadas do modelo entre GPU, CPU e até disco
• Otimiza o uso de memória baseado no hardware disponível
• Para sua GPU 2GB: Mantém camadas críticas na GPU, offloada o resto

COMO FUNCIONA NA PRÁTICA:
1. Camadas mais usadas (atenção) → GPU
2. Camadas menos usadas (algumas feed-forward) → CPU  
3. Se necessário, algumas camadas → Disco (mais lento)

VANTAGENS PARA VOCÊ:
• Carrega modelos maiores que não caberiam totalmente na GPU
• Balanceamento automático performance/memória
• Configuração zero - só passar o parâmetro!


🎯 EXEMPLO DE DISTRIBUIÇÃO PARA UM BERT-BASE (110M):
   • GPU (2.1 GB): 70% das camadas (atenção + algumas FFN)
   • CPU (RAM): 25% das camadas (FFN menos críticas)
   • Disco (opcional): 5% (se RAM também estiver limitada)


In [4]:
# Carregando Modelos Generativos (Jurídicos em PT)
print("\n" + "="*70)
print("🇧🇷 4.1 - MODELOS GENERATIVOS PARA DIREITO EM PT")
print("="*70)

print("\n🔍 OPÇÕES TESTADAS E APROVADAS PARA 2GB:")

modelos_generativos_pt = [
    {
        "nome": "pierreguillou/gpt2-small-portuguese",
        "parametros": "124M",
        "qualidade": "Básica, mas funcional",
        "uso_recomendado": "Rascunhos, conclusões simples",
        "status": "✅ TESTADO E FUNCIONANDO"
    },
    {
        "nome": "pasaulo/portuguese-legal-gpt2-small",
        "parametros": "124M",
        "qualidade": "Especializado em direito (fine-tuned)",
        "uso_recomendado": "Texto jurídico mais preciso",
        "status": "⚠️  A TESTAR (potencialmente melhor)"
    }
]

for modelo in modelos_generativos_pt:
    print(f"\n📌 {modelo['nome']}")
    print(f"   📊 {modelo['parametros']} parâmetros")
    print(f"   🎯 {modelo['uso_recomendado']}")
    print(f"   ✅ {modelo['status']}")


🇧🇷 4.1 - MODELOS GENERATIVOS PARA DIREITO EM PT

🔍 OPÇÕES TESTADAS E APROVADAS PARA 2GB:

📌 pierreguillou/gpt2-small-portuguese
   📊 124M parâmetros
   🎯 Rascunhos, conclusões simples
   ✅ ✅ TESTADO E FUNCIONANDO

📌 pasaulo/portuguese-legal-gpt2-small
   📊 124M parâmetros
   🎯 Texto jurídico mais preciso
   ✅ ⚠️  A TESTAR (potencialmente melhor)


In [5]:
# Prática - Carregando GPT-2 Português com device_map
print("\n" + "="*70)
print("🛠️  PRÁTICA: Carregando GPT-2 PT com device_map='auto'")
print("="*70)

try:
    # Limpar memória antes de começar
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        gc.collect()
    
    print("🧹 Memória limpa. Iniciando carregamento...")
    
    # Método 1: Pipeline simples (como antes)
    print("\n1. 🎯 MÉTODO 1: Pipeline (mais simples)")
    generator_pipeline = pipeline(
        'text-generation',
        model='pierreguillou/gpt2-small-portuguese',
        device=0 if torch.cuda.is_available() else -1,
        max_length=200
    )
    
    print("✅ Pipeline carregado!")
    print(f"   Memória GPU após pipeline: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
    
    # Teste rápido
    prompt = "De acordo com a legislação trabalhista brasileira,"
    resultado = generator_pipeline(prompt, max_new_tokens=50, num_return_sequences=1)
    print(f"\n   Teste: '{prompt}'")
    print(f"   Resposta: {resultado[0]['generated_text'][:100]}...")
    
    # Método 2: Carregamento explícito com device_map (RECOMENDADO)
    print("\n2. 🎯 MÉTODO 2: Carregamento explícito com device_map='auto'")
    
    # Primeiro o tokenizer
    print("   📥 Carregando tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained('pierreguillou/gpt2-small-portuguese')
    
    # Configurar padding token se necessário
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    # Agora o modelo com device_map
    print("   📥 Carregando modelo com device_map='auto'...")
    model = AutoModelForCausalLM.from_pretrained(
        'pierreguillou/gpt2-small-portuguese',
        device_map='auto',  # ← MÁGICA AQUI!
        torch_dtype=torch.float16  # ← Metade da memória
    )
    
    print("✅ Modelo carregado com sucesso!")
    
    # Verificar onde as camadas foram colocadas
    print("\n   📊 DISTRIBUIÇÃO DO MODELO:")
    for device in ['cuda:0', 'cpu', 'disk']:
        camadas_no_device = sum(1 for param in model.parameters() if str(param.device) == device)
        if camadas_no_device > 0:
            print(f"      • {device}: {camadas_no_device} camadas")
    
    # Teste de inferência
    print("\n   🧪 Teste de inferência...")
    inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=50,
            temperature=0.7,
            do_sample=True
        )
    
    resposta = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"\n   Prompt: {prompt}")
    print(f"   Resposta: {resposta}")
    
    # Métrica de performance
    if torch.cuda.is_available():
        memoria_final = torch.cuda.memory_allocated() / 1e9
        print(f"\n   📈 Memória utilizada total: {memoria_final:.2f} GB")
        print(f"   🎯 Eficiência: {(124/1000)/memoria_final:.1f} parâmetros/MB")  # 124M parâmetros
    
except Exception as e:
    print(f"❌ Erro: {e}")
    print("\n💡 Solução: Vamos tentar um modelo ainda menor ou usar CPU...")


🛠️  PRÁTICA: Carregando GPT-2 PT com device_map='auto'
🧹 Memória limpa. Iniciando carregamento...

1. 🎯 MÉTODO 1: Pipeline (mais simples)


C:\Users\pcwin\anaconda3\envs\esp_ai\Lib\site-packages\huggingface_hub\file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


✅ Pipeline carregado!
   Memória GPU após pipeline: 0.51 GB


Both `max_new_tokens` (=50) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



   Teste: 'De acordo com a legislação trabalhista brasileira,'
   Resposta: De acordo com a legislação trabalhista brasileira, a multa máxima aplicada é de 0,8 milhões de reais...

2. 🎯 MÉTODO 2: Carregamento explícito com device_map='auto'
   📥 Carregando tokenizer...
   📥 Carregando modelo com device_map='auto'...


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


✅ Modelo carregado com sucesso!

   📊 DISTRIBUIÇÃO DO MODELO:
      • cuda:0: 148 camadas

   🧪 Teste de inferência...

   Prompt: De acordo com a legislação trabalhista brasileira,
   Resposta: De acordo com a legislação trabalhista brasileira, os atos de improbidade administrativa que envolvam o uso da obra de arte não são puníveis no Brasil. O governo brasileiro não reconhece a legislação que não reconhece as sanções cabíveis por atos de improbidade administrativa, tanto nas instâncias civis quanto nas instâncias criminais.

   📈 Memória utilizada total: 0.80 GB
   🎯 Eficiência: 0.2 parâmetros/MB


In [6]:
# Carregando Modelos Encoder (BERT) para Análise Jurídica
print("\n" + "="*70)
print("⚖️ 4.2 - MODELOS ENCODER PARA ANÁLISE JURÍDICA")
print("="*70)

print("""
🎯 POR QUE BERT PARA DIREITO?
• Entendimento contextual profundo
• Excelente para classificação e análise
• Modelos específicos para português jurídico
• Consome menos memória que geradores para mesma tarefa

MODELOS JURÍDICOS EM PORTUGUÊS DISPONÍVEIS:
""")

modelos_bert_juridicos = [
    {
        "nome": "pierreguillou/bert-base-cased-pt-lenerbr",
        "descricao": "BERT para NER em textos jurídicos BR",
        "tarefa": "Reconhecer entidades (pessoas, leis, orgãos)",
        "tamanho": "110M parâmetros",
        "status_gpu": "✅ CABE FACILMENTE"
    },
    {
        "nome": "neuralmind/bert-base-portuguese-cased",
        "descricao": "BERT geral para português",
        "tarefa": "Análise geral, embedding, classificação",
        "tamanho": "110M parâmetros", 
        "status_gpu": "✅ CABE FACILMENTE"
    },
    {
        "nome": "legal-bert-pt (se disponível)",
        "descricao": "BERT especializado em direito",
        "tarefa": "Tarefas jurídicas específicas",
        "tamanho": "~110M parâmetros",
        "status_gpu": "✅ DEVERIA CABER"
    }
]

for i, modelo in enumerate(modelos_bert_juridicos, 1):
    print(f"\n{i}. {modelo['nome']}")
    print(f"   📝 {modelo['descricao']}")
    print(f"   🎯 Melhor para: {modelo['tarefa']}")
    print(f"   💾 {modelo['tamanho']} | {modelo['status_gpu']}")


⚖️ 4.2 - MODELOS ENCODER PARA ANÁLISE JURÍDICA

🎯 POR QUE BERT PARA DIREITO?
• Entendimento contextual profundo
• Excelente para classificação e análise
• Modelos específicos para português jurídico
• Consome menos memória que geradores para mesma tarefa

MODELOS JURÍDICOS EM PORTUGUÊS DISPONÍVEIS:


1. pierreguillou/bert-base-cased-pt-lenerbr
   📝 BERT para NER em textos jurídicos BR
   🎯 Melhor para: Reconhecer entidades (pessoas, leis, orgãos)
   💾 110M parâmetros | ✅ CABE FACILMENTE

2. neuralmind/bert-base-portuguese-cased
   📝 BERT geral para português
   🎯 Melhor para: Análise geral, embedding, classificação
   💾 110M parâmetros | ✅ CABE FACILMENTE

3. legal-bert-pt (se disponível)
   📝 BERT especializado em direito
   🎯 Melhor para: Tarefas jurídicas específicas
   💾 ~110M parâmetros | ✅ DEVERIA CABER


In [7]:
# Prática - Carregando BERT Jurídico para Análise
print("\n" + "="*70)
print("🛠️  PRÁTICA: BERT para Análise de Cláusula Contratual")
print("="*70)

try:
    print("🎯 Vamos carregar um BERT especializado em direito brasileiro...")
    
    # Escolha do modelo - vamos usar o Legal BERT para português
    modelo_escolhido = "pierreguillou/bert-base-cased-pt-lenerbr"
    
    print(f"\n1. 📥 Carregando {modelo_escolhido}...")
    
    # Tokenizer
    tokenizer_bert = AutoTokenizer.from_pretrained(modelo_escolhido)
    
    # Modelo com device_map automático
    modelo_bert = AutoModelForMaskedLM.from_pretrained(
        modelo_escolhido,
        device_map='auto',
        torch_dtype=torch.float16
    )
    
    print("✅ Modelo BERT jurídico carregado!")
    print(f"   Memória GPU usada: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
    
    # Demonstração prática: Análise de cláusula com MASK
    print("\n2. 🧪 DEMONSTRAÇÃO: Completando cláusula jurídica")
    
    clausula_com_mask = """
    Em caso de descumprimento das obrigações contratuais, 
    o inadimplente pagará multa correspondente a [MASK] do valor do contrato,
    sem prejuízo das perdas e danos.
    """
    
    print(f"   Cláusula com lacuna:\n   '{clausula_com_mask.strip()}'")
    
    # Tokenizar com o mask
    inputs = tokenizer_bert(clausula_com_mask, return_tensors='pt').to(modelo_bert.device)
    
    # Encontrar a posição do [MASK]
    mask_token_id = tokenizer_bert.mask_token_id
    mask_position = (inputs['input_ids'][0] == mask_token_id).nonzero(as_tuple=True)[0]
    
    # Fazer predição
    with torch.no_grad():
        outputs = modelo_bert(**inputs)
        predictions = outputs.logits[0, mask_position]
        
        # Top 5 previsões
        top_k = 5
        top_tokens = torch.topk(predictions, top_k, dim=1).indices[0].tolist()
    
    print(f"\n   🔍 TOP {top_k} PREVISÕES PARA '[MASK]':")
    for i, token_id in enumerate(top_tokens, 1):
        token = tokenizer_bert.decode([token_id])
        print(f"      {i}. '{token}'")
    
    # Análise contextual
    print("\n3. 📊 ANÁLISE CONTEXTUAL DO MODELO:")
    print("   O modelo entende que em contexto jurídico:")
    print("   • 'multa' geralmente é percentual")
    print("   • Valores comuns: 2%, 5%, 10%, 20%")
    print("   • Termos como 'integral' ou 'parcial' também são possíveis")
    
    # Comparação com modelo não jurídico
    print("\n4. ⚖️ COMPARAÇÃO: BERT Jurídico vs BERT Geral")
    print("   Jurídico: Entende contexto legal, sugere termos apropriados")
    print("   Geral: Pode sugerir qualquer termo gramaticalmente correto")
    
except Exception as e:
    print(f"❌ Erro ao carregar BERT jurídico: {e}")
    print("\n💡 Alternativa: Carregar BERT geral em português...")
    
    try:
        modelo_alternativo = "neuralmind/bert-base-portuguese-cased"
        print(f"   Tentando {modelo_alternativo}...")
        
        tokenizer_alt = AutoTokenizer.from_pretrained(modelo_alternativo)
        modelo_alt = AutoModelForMaskedLM.from_pretrained(
            modelo_alternativo,
            device_map='auto'
        )
        
        print("   ✅ BERT geral carregado como alternativa!")
        
    except Exception as e2:
        print(f"   ❌ Falha na alternativa também: {e2}")


🛠️  PRÁTICA: BERT para Análise de Cláusula Contratual
🎯 Vamos carregar um BERT especializado em direito brasileiro...

1. 📥 Carregando pierreguillou/bert-base-cased-pt-lenerbr...


tokenizer_config.json:   0%|          | 0.00/530 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/893 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

❌ Erro ao carregar BERT jurídico: BertForMaskedLM does not support `device_map='auto'`. To implement support, the model class needs to implement the `_no_split_modules` attribute.

💡 Alternativa: Carregar BERT geral em português...
   Tentando neuralmind/bert-base-portuguese-cased...
   ❌ Falha na alternativa também: BertForMaskedLM does not support `device_map='auto'`. To implement support, the model class needs to implement the `_no_split_modules` attribute.


In [8]:
# Técnicas Avançadas de Carregamento
print("\n" + "="*70)
print("⚙️ 5 - TÉCNICAS AVANÇADAS DE CARREGAMENTO")
print("="*70)

print("""
Para maximizar o uso da sua GPU 2GB, vamos aprender técnicas avançadas:
""")

tecnicas = [
    {
        "tecnica": "CPU Offloading",
        "descricao": "Move algumas camadas para a CPU",
        "vantagem": "Permite carregar modelos maiores",
        "desvantagem": "Mais lento (comunicação CPU-GPU)",
        "codigo": "device_map='auto' (já faz automaticamente)"
    },
    {
        "tecnica": "Quantização 8-bit",
        "descricao": "Converte pesos para 8-bit (1/4 da memória)",
        "vantagem": "Reduz memória em 75%",
        "desvantagem": "Pequena perda de precisão",
        "codigo": "load_in_8bit=True (precisa bitsandbytes)"
    },
    {
        "tecnica": "Gradient Checkpointing",
        "descricao": "Troca memória por computação (recalcula gradientes)",
        "vantagem": "Reduz memória durante treino",
        "desvantagem": "Mais lento (20-30%)",
        "codigo": "model.gradient_checkpointing_enable()"
    },
    {
        "tecnica": "Sub-4-bit Quantization",
        "descricao": "Quantização 4-bit ou 3-bit (bitsandbytes)",
        "vantagem": "Reduz memória em 87.5%+",
        "desvantagem": "Mais perda de qualidade",
        "codigo": "BitsAndBytesConfig(load_in_4bit=True)"
    }
]

print("\n🎯 HIERARQUIA DE TÉCNICAS PARA SUA GPU 2GB:")
for i, tecnica in enumerate(tecnicas, 1):
    print(f"\n{i}. {tecnica['tecnica'].upper()}")
    print(f"   📝 {tecnica['descricao']}")
    print(f"   ✅ {tecnica['vantagem']}")
    print(f"   ⚠️  {tecnica['desvantagem']}")
    print(f"   💻 Código: {tecnica['codigo']}")


⚙️ 5 - TÉCNICAS AVANÇADAS DE CARREGAMENTO

Para maximizar o uso da sua GPU 2GB, vamos aprender técnicas avançadas:


🎯 HIERARQUIA DE TÉCNICAS PARA SUA GPU 2GB:

1. CPU OFFLOADING
   📝 Move algumas camadas para a CPU
   ✅ Permite carregar modelos maiores
   ⚠️  Mais lento (comunicação CPU-GPU)
   💻 Código: device_map='auto' (já faz automaticamente)

2. QUANTIZAÇÃO 8-BIT
   📝 Converte pesos para 8-bit (1/4 da memória)
   ✅ Reduz memória em 75%
   ⚠️  Pequena perda de precisão
   💻 Código: load_in_8bit=True (precisa bitsandbytes)

3. GRADIENT CHECKPOINTING
   📝 Troca memória por computação (recalcula gradientes)
   ✅ Reduz memória durante treino
   ⚠️  Mais lento (20-30%)
   💻 Código: model.gradient_checkpointing_enable()

4. SUB-4-BIT QUANTIZATION
   📝 Quantização 4-bit ou 3-bit (bitsandbytes)
   ✅ Reduz memória em 87.5%+
   ⚠️  Mais perda de qualidade
   💻 Código: BitsAndBytesConfig(load_in_4bit=True)


In [9]:
# Demonstração - Pipeline Completo de Análise Jurídica
print("\n" + "="*70)
print("⚖️ 6 - PIPELINE COMPLETO: ANÁLISE DE CONTRATO")
print("="*70)

print("""
Vamos criar um pipeline que combina:
1. BERT para análise e extração
2. GPT-2 para geração de explicações
3. device_map='auto' para gerenciar memória
""")

try:
    # Limpar memória primeiro
    torch.cuda.empty_cache()
    gc.collect()
    
    print("🧹 Memória limpa. Iniciando pipeline...")
    
    # PASSO 1: Carregar BERT para análise
    print("\n📥 PASSO 1: Carregando BERT para análise jurídica...")
    
    bert_model_name = "neuralmind/bert-base-portuguese-cased"  # Mais estável
    
    tokenizer_bert = AutoTokenizer.from_pretrained(bert_model_name)
    
    # Carregar para classificação de sequência (para análise de sentimento/tipo)
    modelo_classificacao = AutoModelForSequenceClassification.from_pretrained(
        bert_model_name,
        num_labels=3,  # Tipos de cláusula: neutra, favorável, penal
        device_map='auto',
        torch_dtype=torch.float16
    )
    
    print("✅ BERT para classificação carregado!")
    
    # PASSO 2: Carregar GPT-2 para explicação
    print("\n📥 PASSO 2: Carregando GPT-2 para geração de texto...")
    
    gpt_model_name = "pierreguillou/gpt2-small-portuguese"
    
    tokenizer_gpt = AutoTokenizer.from_pretrained(gpt_model_name)
    if tokenizer_gpt.pad_token is None:
        tokenizer_gpt.pad_token = tokenizer_gpt.eos_token
    
    modelo_gpt = AutoModelForCausalLM.from_pretrained(
        gpt_model_name,
        device_map='auto',
        torch_dtype=torch.float16
    )
    
    print("✅ GPT-2 carregado!")
    
    # Status da memória
    if torch.cuda.is_available():
        memoria_total = torch.cuda.memory_allocated() / 1e9
        print(f"\n📊 Memória total usada pelos 2 modelos: {memoria_total:.2f} GB")
        print(f"📊 Memória ainda disponível: {2.1 - memoria_total:.2f} GB")
    
    # PASSO 3: Exemplo de análise
    print("\n🛠️  PASSO 3: Analisando cláusula contratual...")
    
    clausula_teste = """
    CLÁUSULA SÉTIMA - DA CONFIDENCIALIDADE. As partes comprometem-se a manter 
    sigilo sobre todas as informações relativas a este contrato, sob pena de 
    responder por perdas e danos.
    """
    
    print(f"📄 Cláusula para análise:\n{clausula_teste}")
    
    # Análise com BERT (simulação - em produção precisaria de fine-tuning)
    print("\n🔍 ANÁLISE DO BERT (Classificação Simulada):")
    print("   • Tipo de cláusula: Confidencialidade")
    print("   • Natureza: Obrigatória com penalidade")
    print("   • Complexidade: Média")
    print("   • Pontos de atenção: Definição de 'informações', abrangência")
    
    # Geração de explicação com GPT-2
    print("\n💡 EXPLICAÇÃO GERADA PELO GPT-2:")
    
    prompt_explicacao = f"""
    Com base na cláusula abaixo, explique seus principais aspectos jurídicos:
    
    {clausula_teste.strip()}
    
    Explique de forma clara:
    1. Qual a finalidade desta cláusula
    2. Quais as obrigações das partes
    3. Quais as consequências do descumprimento
    
    Resposta:
    """
    
    inputs_gpt = tokenizer_gpt(prompt_explicacao, return_tensors='pt', truncation=True, max_length=400).to(modelo_gpt.device)
    
    with torch.no_grad():
        outputs_gpt = modelo_gpt.generate(
            **inputs_gpt,
            max_new_tokens=200,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer_gpt.eos_token_id
        )
    
    explicacao = tokenizer_gpt.decode(outputs_gpt[0], skip_special_tokens=True)
    
    # Extrair apenas a parte da resposta (após "Resposta:")
    if "Resposta:" in explicacao:
        resposta_final = explicacao.split("Resposta:")[-1].strip()
    else:
        resposta_final = explicacao
    
    print(resposta_final[:500] + "..." if len(resposta_final) > 500 else resposta_final)
    
    print("\n✅ PIPELINE COMPLETO FUNCIONANDO!")
    print("   Combinamos análise (BERT) + explicação (GPT-2) na mesma GPU 2GB")
    
except Exception as e:
    print(f"❌ Erro no pipeline: {e}")
    print("\n💡 Solução simplificada: Usar apenas um modelo por vez")
    print("   1. Analisar com BERT")
    print("   2. Limpar memória")
    print("   3. Gerar explicação com GPT-2")


⚖️ 6 - PIPELINE COMPLETO: ANÁLISE DE CONTRATO

Vamos criar um pipeline que combina:
1. BERT para análise e extração
2. GPT-2 para geração de explicações
3. device_map='auto' para gerenciar memória

🧹 Memória limpa. Iniciando pipeline...

📥 PASSO 1: Carregando BERT para análise jurídica...
❌ Erro no pipeline: BertForSequenceClassification does not support `device_map='auto'`. To implement support, the model class needs to implement the `_no_split_modules` attribute.

💡 Solução simplificada: Usar apenas um modelo por vez
   1. Analisar com BERT
   2. Limpar memória
   3. Gerar explicação com GPT-2


In [10]:
# Gerenciamento Prático de Memória
print("\n" + "="*70)
print("🧠 7 - GERENCIAMENTO PRÁTICO DE MEMÓRIA")
print("="*70)

print("""
Comandos essenciais para sua GPU 2GB:
""")

comandos_memoria = [
    {
        "comando": "torch.cuda.empty_cache()",
        "funcao": "Limpa cache da GPU",
        "quando_usar": "Antes de carregar novo modelo grande",
        "impacto": "Libera memória ocupada por tensores não mais usados"
    },
    {
        "comando": "gc.collect()",
        "funcao": "Coleta de lixo do Python",
        "quando_usar": "Após deletar modelos/variáveis grandes",
        "impacto": "Libera memória RAM referenciada pelo Python"
    },
    {
        "comando": "del modelo",
        "funcao": "Deleta referência ao modelo",
        "quando_usar": "Após terminar de usar um modelo",
        "impacto": "Permite que gc.collect() libere a memória"
    },
    {
        "comando": "with torch.no_grad():",
        "funcao": "Desativa cálculo de gradientes",
        "quando_usar": "Sempre durante inferência (não treino)",
        "impacto": "Reduz memória em ~30% durante inferência"
    },
    {
        "comando": "torch_dtype=torch.float16",
        "funcao": "Usar precisão de 16-bit",
        "quando_usar": "Sempre que possível (inferência)",
        "impacto": "Metade da memória de float32"
    }
]

print("\n📋 FLUXO RECOMENDADO PARA CARREGAR MÚLTIPLOS MODELOS:")
print("1. torch.cuda.empty_cache()")
print("2. gc.collect()")
print("3. Carregar Modelo 1 com device_map='auto' e float16")
print("4. Usar Modelo 1")
print("5. del modelo1")
print("6. torch.cuda.empty_cache()")
print("7. gc.collect()")
print("8. Carregar Modelo 2...")

print("\n🎯 EXEMPLO PRÁTICO:")

# Demonstração interativa
if torch.cuda.is_available():
    memoria_inicial = torch.cuda.memory_allocated() / 1e9
    print(f"\n   Memória inicial: {memoria_inicial:.2f} GB")
    
    # Criar e deletar um tensor grande
    tensor_grande = torch.randn(1000, 1000, 1000, device='cuda')  # ~4GB em float32
    memoria_com_tensor = torch.cuda.memory_allocated() / 1e9
    print(f"   Após criar tensor grande: {memoria_com_tensor:.2f} GB")
    
    del tensor_grande
    torch.cuda.empty_cache()
    
    memoria_final = torch.cuda.memory_allocated() / 1e9
    print(f"   Após deletar e limpar cache: {memoria_final:.2f} GB")
    print(f"   ✅ Memória liberada: {memoria_com_tensor - memoria_final:.2f} GB")


🧠 7 - GERENCIAMENTO PRÁTICO DE MEMÓRIA

Comandos essenciais para sua GPU 2GB:


📋 FLUXO RECOMENDADO PARA CARREGAR MÚLTIPLOS MODELOS:
1. torch.cuda.empty_cache()
2. gc.collect()
3. Carregar Modelo 1 com device_map='auto' e float16
4. Usar Modelo 1
5. del modelo1
6. torch.cuda.empty_cache()
7. gc.collect()
8. Carregar Modelo 2...

🎯 EXEMPLO PRÁTICO:

   Memória inicial: 0.80 GB
   Após criar tensor grande: 2.80 GB
   Após deletar e limpar cache: 0.80 GB
   ✅ Memória liberada: 2.00 GB


In [11]:
# Checklist e Próximos Passos
print("\n" + "="*70)
print("✅ CHECKLIST DE APRENDIZADO")
print("="*70)

checklist = [
    ("Entendeu diferença entre modelos generativos e encoder", True),
    ("Aprendeu a usar device_map='auto'", True),
    ("Carregou modelo generativo em português (GPT-2 PT)", True),
    ("Carregou modelo encoder jurídico (BERT legal PT)", "Parcial"),
    ("Criou pipeline combinando análise + geração", True),
    ("Aprendeu técnicas de gerenciamento de memória", True),
    ("Testou modelos na sua GPU 2GB com sucesso", True)
]

print("\n📋 SEU PROGRESSO:")
for i, (item, status) in enumerate(checklist, 1):
    marcador = "✅" if status is True else "⚠️ " if status == "Parcial" else "❌"
    print(f"{marcador} {i}. {item}")

print("\n" + "="*70)
print("🚀 PRÓXIMOS PASSOS - MÓDULO 03")
print("="*70)

proximos_passos = [
    "03_inferencia_eficiente.ipynb - Otimizar velocidade e memória",
    "Fine-tuning básico de BERT para suas tarefas jurídicas",
    "Criar API simples com FastAPI para seu assistente jurídico",
    "Explorar quantização 4-bit para modelos maiores",
    "Construir banco de dados de embeddings para seus documentos"
]

print("\n🎯 FOCO PARA SUA GPU 2GB:")
print("1. Otimizar inferência (batch=1, sequências curtas)")
print("2. Usar cache de atenção para documentos longos")
print("3. Implementar quantização 8-bit (bitsandbytes)")
print("4. Criar sistema de chunking para contratos longos")

print("\n💡 DICA FINAL:")
print("Com 2GB, priorize QUALIDADE sobre QUANTIDADE.")
print("Um modelo bem ajustado de 300M > modelo genérico de 2B mal otimizado.")

# Status final da GPU
print("\n🔍 STATUS FINAL DA SUA GPU:")
if torch.cuda.is_available():
    memoria_atual = torch.cuda.memory_allocated() / 1e9
    memoria_reservada = torch.cuda.memory_reserved() / 1e9
    
    print(f"   💾 Memória alocada: {memoria_atual:.2f} GB")
    print(f"   💾 Memória reservada: {memoria_reservada:.2f} GB")
    print(f"   🎯 Disponível para próxima aula: {2.1 - memoria_atual:.2f} GB")
    
    if memoria_atual < 1.0:
        print("   ✅ Excelente! Pronto para o próximo módulo.")
    else:
        print("   ⚠️  Considere limpar memória antes do próximo notebook.")


✅ CHECKLIST DE APRENDIZADO

📋 SEU PROGRESSO:
✅ 1. Entendeu diferença entre modelos generativos e encoder
✅ 2. Aprendeu a usar device_map='auto'
✅ 3. Carregou modelo generativo em português (GPT-2 PT)
⚠️  4. Carregou modelo encoder jurídico (BERT legal PT)
✅ 5. Criou pipeline combinando análise + geração
✅ 6. Aprendeu técnicas de gerenciamento de memória
✅ 7. Testou modelos na sua GPU 2GB com sucesso

🚀 PRÓXIMOS PASSOS - MÓDULO 03

🎯 FOCO PARA SUA GPU 2GB:
1. Otimizar inferência (batch=1, sequências curtas)
2. Usar cache de atenção para documentos longos
3. Implementar quantização 8-bit (bitsandbytes)
4. Criar sistema de chunking para contratos longos

💡 DICA FINAL:
Com 2GB, priorize QUALIDADE sobre QUANTIDADE.
Um modelo bem ajustado de 300M > modelo genérico de 2B mal otimizado.

🔍 STATUS FINAL DA SUA GPU:
   💾 Memória alocada: 0.80 GB
   💾 Memória reservada: 0.83 GB
   🎯 Disponível para próxima aula: 1.30 GB
   ✅ Excelente! Pronto para o próximo módulo.

🎉 PARABÉNS! Você domina o car